In [1]:
import numpy as np

from networks import FiniteFieldPiNetNetworkLinear
from criterions import FiniteFieldMSELoss
from datasets import load_all_data_mnist
from utils import create_batch_data, to_real_domain

In [2]:
BATCH_SIZE = 128
EPOCH = 1
LR = 7
PRINT = 10
MODE = 1
PRIME = 2**26 - 5
QUANTIZATION_WEIGHT = 8
QUANTIZATION_INPUT = 8
QUANTIZATION_BATCH_SIZE = 7

In [3]:
if MODE == 1:
    model = FiniteFieldPiNetNetworkLinear(QUANTIZATION_WEIGHT, PRIME, QUANTIZATION_INPUT)
    flatten = True
else:
    model = None
    flatten = True
criterion = FiniteFieldMSELoss(PRIME, QUANTIZATION_WEIGHT, QUANTIZATION_BATCH_SIZE)

In [4]:
# data fetching
load_path = '../../data'
train_data, train_label, test_data, test_label = load_all_data_mnist(load_path, QUANTIZATION_INPUT, QUANTIZATION_WEIGHT, PRIME, flatten=flatten)
train_data, train_label, test_data, test_label = create_batch_data(train_data, train_label, test_data, test_label, BATCH_SIZE)

In [ ]:
for epoch in range(EPOCH):
    tot_loss = 0
    for train_idx, (train_data_batch, train_label_batch) in enumerate(zip(train_data, train_label)):
        # train
        preds = model.forward(train_data_batch)

        loss = criterion.forward(preds, train_label_batch)
        tot_loss += loss
        propagated_error = criterion.error_derivative()

        model.backprop(propagated_error)
        model.optimize(LR)

        print('idx: {}, loss: {}'.format(train_idx + 1, loss))

        if (train_idx + 1) % PRINT == 0:
            tot_acc = 0
            tot_sample = 0
            for train_acc_idx, (test_data_batch, test_label_batch) in enumerate(zip(test_data, test_label)):
                # train accuracy
                preds = model.forward(test_data_batch)
                real_preds = to_real_domain(preds, QUANTIZATION_WEIGHT, PRIME)
                pred_args = np.argmax(real_preds, axis=1)

                tot_acc += np.count_nonzero(pred_args == test_label_batch)
                tot_sample += test_data_batch.shape[0]
            accuracy = tot_acc / tot_sample
            if train_idx != 0:
                tot_loss = tot_loss / PRINT
            print('epoch: {}, idx: {}, accuracy: {}, loss: {}'.format(epoch + 1, train_idx + 1, accuracy, tot_loss))
            tot_loss = 0

idx: 1, loss: 2.482264399528504
idx: 2, loss: 2.297526836395264
idx: 3, loss: 1.7566620111465454
idx: 4, loss: 1.3414357900619507
idx: 5, loss: 1.159978985786438
idx: 6, loss: 1.1256710290908811
idx: 7, loss: 1.0051747560501096
idx: 8, loss: 0.9812606573104857
idx: 9, loss: 1.0735796689987183
idx: 10, loss: 0.9512060880661012
epoch: 1, idx: 10, accuracy: 0.499, loss: 1.4174760222434997
idx: 11, loss: 1.2107012271881104
idx: 12, loss: 1.5241149663925169
idx: 13, loss: 0.9501550197601317
idx: 14, loss: 0.6984649896621704
idx: 15, loss: 0.7994083166122435
idx: 16, loss: 0.7833993434906007
idx: 17, loss: 0.6476274728775024
idx: 18, loss: 0.6904256343841554
idx: 19, loss: 0.6776821613311766
idx: 20, loss: 0.7115340232849121
epoch: 1, idx: 20, accuracy: 0.6501, loss: 0.8693513154983521
idx: 21, loss: 0.6352277994155885
idx: 22, loss: 0.7486178874969482
idx: 23, loss: 0.6487970352172853
idx: 24, loss: 0.7018412351608276
idx: 25, loss: 0.6386617422103883
idx: 26, loss: 0.6010475158691406
idx: 